In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sc = spark.sparkContext

In [0]:
schema = StructType([
    StructField("ID", IntegerType(), True)
    ,StructField("Version", FloatType(), True)
    ,StructField("Details", StringType(), True)
])

df = spark.createDataFrame(sc.emptyRDD(),schema)
df.show()

+---+-------+-------+
 ID|Version|Details|
+---+-------+-------+
+---+-------+-------+

In [0]:
#add details into the empty df 
df = spark.createDataFrame([
    [1, 0.1, """{"name": "abc", 
                "age": 15,
                "occupation": "student"}"""]
    ,[2, 0.1, """{"name": "xyz",
                  "age": 45,
                  "occupation": "entrepreneur"}"""]
    ,[3, 0.1, """{"name": "ghi", 
                    "age": 50,
                    "occupation": "doctor"}"""]
], schema)
display(df)

ID,Version,Details
1,0.1,"{""name"": ""abc"", ""age"": 15, ""occupation"": ""student""}"
2,0.1,"{""name"": ""xyz"", ""age"": 45, ""occupation"": ""entrepreneur""}"
3,0.1,"{""name"": ""ghi"", ""age"": 50, ""occupation"": ""doctor""}"


In [0]:
#write to schema
df.write.mode("overwrite").saveAsTable("default.temp_table", format='delta')

In [0]:
parameters = {"ID": 4
               ,"Version": 0.1, "Details": {"name": "def", "age": 25, "occupation": "engineer"}}
parameters

Out[19]: {'ID': 4,
 'Version': 0.1,
 'Details': {'name': 'def', 'age': 25, 'occupation': 'engineer'}}

In [0]:
if parameters:
    parameters = eval('parameters')
else:
    raise ValueError("please check the json")

if isinstance(parameters,list):
    for item in parameters:
        item_ = str(parameters['Details']).replace('''\'''', '''"''')
        spark.sql(f"""insert into default.temp_table values({item['ID']}, {item['Version']}, '{item_}')""")
elif isinstance(parameters, dict):
    param_ = str(parameters['Details']).replace('''\'''', '''"''')
    spark.sql(f"""insert into default.temp_table values({parameters['ID']}, {parameters['Version']}, '{param_}')""")

In [0]:
%sql 
select * from default.temp_table

ID,Version,Details
4,0.1,"{""ID"": 4, ""Version"": 0.1, ""Details"": {""name"": ""def"", ""age"": 25, ""occupation"": ""engineer""}}"
2,0.1,"{""name"": ""xyz"", ""age"": 45, ""occupation"": ""entrepreneur""}"
3,0.1,"{""name"": ""ghi"", ""age"": 50, ""occupation"": ""doctor""}"
1,0.1,"{""name"": ""abc"", ""age"": 15, ""occupation"": ""student""}"
4,0.1,"{""name"": ""def"", ""age"": 25, ""occupation"": ""engineer""}"


In [0]:
f